In [ ]:
%matplotlib inline

import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn import preprocessing
from scipy import ndimage
import scipy
import skimage
#from medpy.filter import IntensityRangeStandardization
import time, os
import shutil, pathlib
from functools import partial

In [2]:
from pathlib import Path
p=Path("subjects/")
file_img = list(p.rglob('*pp_t1w.nii.gz'))
file_mask = list(p.rglob('*brainmask.nii.gz'))
file_all = list(p.rglob('*.gz'))

In [3]:
def read_all_images():
    brain_list = []
    for skull_path, mask_path in list(zip(file_img, file_mask)): #gre skozi vse slike
        #make nparrays from images in file
        skull_sitk = sitk.ReadImage(str(skull_path), sitk.sitkFloat32)
        skull_array = sitk.GetArrayFromImage(skull_sitk)
        mask_sitk = sitk.ReadImage(str(mask_path), sitk.sitkFloat32)
        mask_array = sitk.GetArrayFromImage(mask_sitk)
        brain_array = skull_array * mask_array #mask just the brain
        brain_list.append(brain_array) #get a list of brain images
    return brain_list

In [4]:
def rotMat(alpha, beta, gamma):
    def rotation_x(alpha):
        cos, sin = np.cos(alpha), np.sin(alpha)
        oMatRotx = np.array([[1, 0, 0, 0], [0, cos, -sin, 0], [0, sin, cos, 0], [0, 0, 0, 1]])
        return oMatRotx
    def rotation_y(alpha):
        cos, sin = np.cos(alpha), np.sin(alpha)
        oMatRoty = np.array([[cos, 0, sin, 0], [0, 1, 0, 0], [-sin, 0, cos, 0], [0, 0, 0, 1]])
        return oMatRoty
    def rotation_z(alpha):
        cos, sin = np.cos(alpha), np.sin(alpha)
        oMatRotz = np.array([[cos, -sin, 0, 0], [sin, cos, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        return oMatRotz
    oMatRotx = rotation_x(alpha)
    oMatRoty = rotation_y(beta)
    oMatRotz = rotation_z(gamma)
    return oMatRotx.dot(oMatRoty).dot(oMatRotz)


In [5]:
def transAffine3D(iScale=(1, 1, 1), iTrans=(0, 0, 0), iRot=(0, 0, 0), iShear=(0, 0, 0, 0, 0, 0)):
    oMatScale = np.array([[iScale[0], 0, 0, 0],
                        [0, iScale[1], 0, 0],
                        [0, 0, iScale[2], 0],
                        [0, 0, 0, 1]])
    oMatTrans = np.array([[1, 0, 0, iTrans[0]],
                            [0, 1, 0, iTrans[1]],
                            [0, 0, 1, iTrans[2]],
                            [0, 0, 0, 1]])
    oMatShear = np.array([[1, iShear[0], iShear[1], 0],
                         [iShear[2], 1, iShear[5], 0],
                         [iShear[3], iShear[4], 1, 0],
                         [0, 0, 0, 1]])
    normalized_angles = [degree * np.pi / 180.0 for degree in iRot]
    oRot = rotMat(*normalized_angles)
    oMat3D = oMatTrans.dot(oMatShear).dot(oRot).dot(oMatScale)
    return oMat3D

In [6]:
def randomly_transfor_img(img):
    scale = np.random.rand(3)/5 + 0.9
    trans = (0,0,0)
    rot = np.random.rand(3)*10 - 5
    shear = np.random.rand(6)/5 - 0.1
    matrix = transAffine3D(iScale=scale, iTrans= trans, iRot= rot, iShear=shear)
    pad_list = np.pad(img, (15,), 'constant', constant_values=(0,))
    return scipy.ndimage.affine_transform(pad_list, matrix.T)

In [9]:
np.random.rand(3)/5 + 0.9

array([0.98972354, 0.9559916 , 1.00058543])

In [17]:
1/5-0.1

0.1

In [7]:
from random import choice
def transform_and_save(brain_list, number_of_outputs):
    shutil.rmtree('working_data', ignore_errors=True)
    os.makedirs('working_data')
    for i in range(number_of_outputs):
        np.save("working_data/{}".format(i), randomly_transfor_img(choice(brain_list)))

In [8]:
def main():
    brain_list = read_all_images()
    transform_and_save(brain_list, 20)
    #for i, brain in enumerate(brain_list):
    #    np.save('working_data/orig_{}'.format(i), brain)

In [9]:
main()

In [ ]:
impo